In [1]:
%load_ext autoreload
%autoreload 2
import sys, os
from os.path import expanduser
## actions required!!!!!!!!!!!!!!!!!!!! change your folder path 
path = "~/Documents/G3_2/regime-identification"
path = expanduser(path)
sys.path.append(path)

path_file = f"~/data/G3_2/regime-identification/simulation"
path_file = expanduser(path_file)
path_data = f"{path_file}/data"
path_estimation = f"{path_file}/estimation"
path_score = f"{path_file}/score"

In [67]:
import numpy as np
from sklearn.metrics import roc_auc_score
from hmmlearn.hmm import GaussianHMM
from tqdm import trange, tqdm
import logging
logging.basicConfig(level=logging.WARNING+1)

from numpy.random import RandomState
random_state = RandomState(0)

In [127]:
from regime.simulation_helper import *
from regime.jump import _sort_centers_by_first_feature
from regime.cluster_utils import *

from sklearn.utils import check_random_state
from sklearn.cluster import kmeans_plusplus

# HMM estimation

In this notebok we estimated the simulated sequence by HMMs.

In [6]:
scale_lst = ["daily", "weekly", "monthly"]
n_s_lst = [250, 500, 1000]
key_data_list = [f"{scale}_{n_s}" for scale in scale_lst for n_s in n_s_lst]

n_buffer, n_t, n_c = 20, 1024, 2

In [5]:
load_hardy_params("daily")

(array([[ 0.000615],
        [-0.000785]]),
 array([[6.02045e-05],
        [3.02642e-04]]),
 array([[0.99788424, 0.00211576],
        [0.01198171, 0.98801829]]))

In [52]:
hmm_model = GaussianHMM(n_components=n_c, init_params="sc", min_covar=1e-6, covars_prior=1e-6, random_state=random_state, n_iter=300, tol=1e-4, )

In [13]:
key_data = "daily_1000"
n_s = 1000
Xs = np.load(f"{path_data}/X_raw_{key_data}.npy")[:, -n_s:]

In [119]:
Zs = np.load(f"{path_data}/Z_{key_data}.npy")#[:, -n_s:]

In [120]:
Zs.shape

(1024, 1000)

In [14]:
Xs.shape

(1024, 1000, 1)

In [59]:
def init_k_means_plusplus(X, n_c, n_init=10, random_state=None):
    """
    initialize the centers, by k-means++, for n_init times.
    """
    random_state = check_random_state(random_state)
    #
    init = np.empty((n_init, n_c, X.shape[1]))  
    for i_ in range(n_init):
        init[i_] = kmeans_plusplus(X, n_c, random_state=random_state)[0]
    # sort the centers by the first element (assumed to be some returns), in descending order, so that crash periods in regimes w/ higher no.
    return _sort_centers_by_first_feature(init)

In [79]:
def fit_HMM_with_init(hmm_model, X, means_init_, transmat_init):
    # initialize
    hmm_model.means_ = means_init_
    hmm_model.transmat_ = transmat_init
    # fit model
    hmm_model.fit(X)
    return 

In [104]:
def train_HMM_batch_data(hmm_model, Xs, n_init=10, transmat_init = np.array([[.9, .1], [.1, .9]]), random_state=None):
    """
    Train an HMM model on a batch of data. 
    Centers initialized 10 times by k-means++. the best model is retained.
    return estimated means, covars, labels and proba
    """
    random_state = check_random_state(random_state)
    n_t, n_s, n_f = Xs.shape
    n_c = hmm_model.n_components
    means_arr = np.empty((n_t, n_c, n_f)); covars_arr = np.empty((n_t, n_c, n_f, n_f))
    labels_arr = np.empty((n_t, n_s)); proba_arr = np.empty((n_t, n_s, n_c))
    for i_trial in trange(n_t):
        X = Xs[i_trial]
        means_init = init_k_means_plusplus(X, n_c, n_init, random_state=random_state)
        best_score = -np.inf
        for i_init in range(n_init):
            fit_HMM_with_init(hmm_model, X, means_init[i_init], transmat_init)
            score = hmm_model.score(X)
            # print(f"{i_init}: {score}")
            if score > best_score:
                best_idx = i_init
                best_score = score
        # print(best_idx)
        fit_HMM_with_init(hmm_model, X, means_init[best_idx], transmat_init)
        means_arr[i_trial] = hmm_model.means_
        covars_arr[i_trial] = hmm_model.covars_
        labels_arr[i_trial] = hmm_model.predict(X)
        proba_arr[i_trial] = hmm_model.predict_proba(X)     
    return means_arr, covars_arr, labels_arr, proba_arr

In [118]:
means_arr, covars_arr, labels_arr, proba_arr=train_HMM_batch_data(hmm_model, Xs, random_state=random_state)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1024/1024 [02:52<00:00,  5.92it/s]


In [129]:
def align_labels_proba_by_accuracy_(Zs_true, labels_arr, n_c):
    """
    In a clustering problem, any permutation of the labels is a vaid clustering result. 
    we find the best permutation for each trial. Here best refers to the highest accuracy.
    
    Parameters:
    ---------------------
    labels_arr: arr of shape (n_t, n_s)
    """
    n_t, n_s = labels_arr.shape
    # all the perms
    all_perms = generate_all_perms_as_arr(n_c)
    # all the possible perms of labels
    labels_all_perms = permute_labels(labels_arr, all_perms)
    # score accuracy for each perm
    acc_all_perms = scorer_batch(accuracy_score, Zs_true, labels_all_perms, has_params=True) # of shape (n_t, n_p)
    # best perm for each trial 
    best_perm_idx = acc_all_perms.argmax(-1) # shape (n_t,)
    # take the corresponding perm for labels
    labels_arr_new = np.take_along_axis(labels_all_perms, best_perm_idx[:, np.newaxis, np.newaxis], axis=-1).squeeze(axis=-1)
    # do the same for proba_
    best_perm = all_perms[best_perm_idx]
    return labels_arr_new, best_perm
    proba_arr_new = np.take_along_axis(proba_arr, best_perm[:, np.newaxis, :], axis=-1)
    # proba_arr_new = proba_arr[np.arange(n_t)[:, np.newaxis, np.newaxis], np.arange(n_s)[np.newaxis, :, np.newaxis], best_perm[:, np.newaxis, :]]
    return proba_arr_new, labels_arr_new

In [130]:
labels_arr_new, best_perm = align_labels_proba_by_accuracy_(Zs, labels_arr, n_c)

In [142]:
proba_arr_new = align_proba_arr(proba_arr, best_perm)

In [146]:
roc_auc_arr = scorer_batch(roc_auc_score, Zs, proba_arr_new[..., 1], idx_subset=get_idx_have_all_clusters(Zs, n_c))

In [148]:
np.nanmean(roc_auc_arr)

0.9837791826688049

In [138]:
def align_proba_arr(proba_arr, best_perm):
    return np.take_along_axis(proba_arr, best_perm[:, np.newaxis, :], axis=-1)
    
def align_means_covars(means_arr, covars_arr, best_perm):
    n_t, n_s, n_f = means_arr.shape
    return means_arr[np.arange(n_t)[:, np.newaxis], best_perm], covars_arr[np.arange(n_t)[:, np.newaxis], best_perm]

In [139]:
means_arr_aligned, covars_arr_aligned = align_means_covars(means_arr, covars_arr, best_perm)

In [140]:
means_arr_aligned.shape

(1024, 2, 1)

In [141]:
means_arr_aligned.mean(0)

array([[ 0.00059545],
       [-0.00064935]])

In [132]:
acc = scorer_batch(accuracy_each_cluster, Zs, labels_arr_new)

In [135]:
np.nanmean(acc, 0).mean()

0.952070423057263

In [137]:
best_perm

array([[0, 1],
       [0, 1],
       [1, 0],
       ...,
       [1, 0],
       [0, 1],
       [0, 1]])

In [121]:
labels_arr.shape

(1024, 1000)

In [108]:
means_arr.shape

(2, 2, 1)

In [115]:
test_arr = random_state.randn(3, 2, 4)
test_arr

array([[[ 0.76103773,  0.12167502,  0.44386323,  0.33367433],
        [ 1.49407907, -0.20515826,  0.3130677 , -0.85409574]],

       [[-2.55298982,  0.6536186 ,  0.8644362 , -0.74216502],
        [ 2.26975462, -1.45436567,  0.04575852, -0.18718385]],

       [[ 1.53277921,  1.46935877,  0.15494743,  0.37816252],
        [-0.88778575, -1.98079647, -0.34791215,  0.15634897]]])

In [116]:
best_idx_perm = np.array([[0, 1], [1, 0], [1, 0]])

In [113]:
best_idx_perm

array([[0, 1],
       [1, 0],
       [1, 0]])

In [117]:
test_arr[np.arange(3)[:, np.newaxis], best_idx_perm]

array([[[ 0.76103773,  0.12167502,  0.44386323,  0.33367433],
        [ 1.49407907, -0.20515826,  0.3130677 , -0.85409574]],

       [[ 2.26975462, -1.45436567,  0.04575852, -0.18718385],
        [-2.55298982,  0.6536186 ,  0.8644362 , -0.74216502]],

       [[-0.88778575, -1.98079647, -0.34791215,  0.15634897],
        [ 1.53277921,  1.46935877,  0.15494743,  0.37816252]]])

In [ ]:
means_arr

In [74]:
Xs[:2].shape

(2, 1000, 1)

In [68]:
hmm.get_params()

{'algorithm': 'viterbi',
 'covariance_type': 'diag',
 'covars_prior': 1e-06,
 'covars_weight': 1,
 'implementation': 'log',
 'init_params': 'sc',
 'means_prior': 0,
 'means_weight': 0,
 'min_covar': 1e-06,
 'n_components': 2,
 'n_iter': 300,
 'params': 'stmc',
 'random_state': RandomState(MT19937) at 0x1464E7340,
 'startprob_prior': 1.0,
 'tol': 0.0001,
 'transmat_prior': 1.0,
 'verbose': False}

In [54]:
hmm_model.means = means_init[0]
hmm_model.transmat_ = generate_2d_TPM(.9, .9)

In [55]:
hmm_model.fit(Xs[0])

GaussianHMM(covars_prior=1e-06, init_params='sc', min_covar=1e-06,
            n_components=2, n_iter=300,
            random_state=RandomState(MT19937) at 0x1464E7340, tol=0.0001)

In [61]:
hmm_model.score_samples(Xs[0])

(3150.2150731016895,
 array([[3.30789689e-08, 9.99999967e-01],
        [4.15596571e-04, 9.99584403e-01],
        [6.19204955e-04, 9.99380795e-01],
        ...,
        [4.97151215e-03, 9.95028488e-01],
        [6.92505088e-03, 9.93074949e-01],
        [1.09651671e-02, 9.89034833e-01]]))

In [62]:
hmm_model.score(Xs[0])

3150.2150731016895

In [63]:
hmm_model.means

array([[ 0.00181927],
       [-0.01728237]])

In [66]:
hmm_model.transmat_

array([[0.98265955, 0.01734045],
       [0.00832298, 0.99167702]])

In [65]:
load_hardy_params("daily")

(array([[ 0.000615],
        [-0.000785]]),
 array([[6.02045e-05],
        [3.02642e-04]]),
 array([[0.99788424, 0.00211576],
        [0.01198171, 0.98801829]]))

In [58]:
hmm_model.covars_.shape

(2, 1, 1)

In [48]:
hmm_model.transmat_

array([[0.99168301, 0.00831699],
       [0.01732639, 0.98267361]])

In [47]:
hmm_model.score(Xs[0])

3150.215073225688

In [24]:
hmm_model.transmat_

array([[0.1, 0.9],
       [0.9, 0.1]])

In [21]:
means_init = init_k_means_pluspus(Xs[0], 2, 10)

In [ ]:
def fit_hmm_model()

In [ ]:
hmm_model.means_ = 
hmm_model.transmat_ = 

In [ ]:
hmm_model.fit()